In [3]:
from main.model import Model
import datetime as dt
from dateutil.relativedelta import relativedelta

ts_start = dt.datetime(2024, 1, 1, 0, 0, 0)
time_step = relativedelta(minutes=15)

sim = Model()
results = sim.simulate(ts_start, time_step)

In [4]:
results
# results.dtypes
# results['commod']['hunger']
commodities = results['commod'].copy()
commodities.index = results['clock']['ts']
state = results['meta'][' state'] # TODO remove leading space " state"

In [29]:
import pandas as pd

output = pd.DataFrame(index = results.index)
output['state'] = results['meta'][' state']
output['group'] = (output['state'] != output['state'].shift()).cumsum()
output['timestamp'] = results['clock']['ts']
schedule = output.groupby('group').agg(
    start_idx=('state', lambda x:x.index[0]),
    end_idx=('state', lambda x:x.index[-1]),
    start_time=('timestamp', 'first'),
    end_time=('timestamp', 'last'),
    value=('state', 'first')
)
schedule

,start_idx,end_idx,start_time,end_time,value
group,,,,,
1,0,36,2024-01-01 00:00:00,2024-01-01 09:00:00,eat
2,37,52,2024-01-01 09:15:00,2024-01-01 13:00:00,work
3,53,86,2024-01-01 13:15:00,2024-01-01 21:30:00,sleep
4,87,87,2024-01-01 21:45:00,2024-01-01 21:45:00,eat
5,88,94,2024-01-01 22:00:00,2024-01-01 23:30:00,watch tv
...,...,...,...,...,...
603,7134,7145,2024-03-15 07:30:00,2024-03-15 10:15:00,sleep
604,7146,7156,2024-03-15 10:30:00,2024-03-15 13:00:00,work
605,7157,7160,2024-03-15 13:15:00,2024-03-15 14:00:00,watch tv


In [40]:
date_start = dt.datetime(2024, 2, 5, 0, 0, 0)
date_end = date_start + relativedelta(days=1)
schedule[(schedule['start_time'] <= date_end) & (schedule['end_time'] >= date_start)]

,start_idx,end_idx,start_time,end_time,value
group,,,,,
272,3331,3367,2024-02-04 16:45:00,2024-02-05 01:45:00,eat
273,3368,3396,2024-02-05 02:00:00,2024-02-05 09:00:00,sleep
274,3397,3412,2024-02-05 09:15:00,2024-02-05 13:00:00,work
275,3413,3416,2024-02-05 13:15:00,2024-02-05 14:00:00,watch tv
276,3417,3432,2024-02-05 14:15:00,2024-02-05 18:00:00,work
277,3433,3434,2024-02-05 18:15:00,2024-02-05 18:30:00,eat
278,3435,3436,2024-02-05 18:45:00,2024-02-05 19:00:00,watch tv
279,3437,3437,2024-02-05 19:15:00,2024-02-05 19:15:00,eat
280,3438,3440,2024-02-05 19:30:00,2024-02-05 20:00:00,watch tv
